In [35]:
import pandas as pd
import numpy as np

# Load the CSV file
df = pd.read_csv('inflation_annual_percent.csv')
df.fillna(0, inplace=True)
df=df.T
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

df.head()

country,Aruba,Afghanistan,Angola,Albania,Andorra,United Arab Emirates,Argentina,Armenia,American Samoa,Antigua and Barbuda,...,St. Vincent and the Grenadines,Venezuela,Virgin Islands (U.S.),Vietnam,Vanuatu,Samoa,Yemen,South Africa,Zambia,Zimbabwe
1961,0,0,0,0,0,0,20.3,0,0,0,...,2.5,2.01,0,0,0,0,0,1.35,−3.66,−2.04
1962,0,0,0,0,0,0,28.9,0,0,0,...,0.0537,0.665,0,0,0,0,0,0.383,2.09,0.47
1963,0,0,0,0,0,0,25.6,0,0,0,...,0.686,4.92,0,0,0,0,0,3.27,0.404,−2.35
1964,0,0,0,0,0,0,28.8,0,0,0,...,3.85,−0.366,0,0,0,0,0,1.99,4.08,6.14
1965,0,0,0,0,0,0,21.2,0,0,0,...,1.47,2.17,0,0,0,0,0,2.95,10.6,2.7


In [36]:
df.drop(df.columns.difference(['country', 'Pakistan']), axis=1, inplace=True)
df = df.replace('−', '-', regex=True)
df.head()

country,Pakistan
1961,3.65
1962,0.16
1963,-1.15
1964,4.49
1965,3.17


In [37]:
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model

# Define the input shape
input_shape = (60,)

# Define the encoding dimension
encoding_dim = 3

# Define the input layer
input_layer = Input(shape=input_shape)

# Define the encoding layer
encoding_layer = Dense(encoding_dim, activation='relu')(input_layer)

# Define the decoding layer
decoding_layer = Dense(60, activation='sigmoid')(encoding_layer)

# Define the autoencoder model
autoencoder = Model(input_layer, decoding_layer)

# Define the encoder model
encoder = Model(input_layer, encoding_layer)

# Define the decoder model
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Prepare the data
data = df.to_numpy().flatten().astype('float32')
data = np.reshape(data, (1, -1))  # Reshape to 2D tensor

# Train the autoencoder model
autoencoder.fit(data, data, epochs=50)

# Use the encoder to get the encoded representation of the data
encoded_data = encoder.predict(data)

# Use the decoder to get the reconstructed data from the encoded data
decoded_data = decoder.predict(encoded_data)

# Print the reconstructed data
print(decoded_data)


Epoch 1/50
1/1 [==============================] - 0s 491ms/step - loss: 5.7287
Epoch 2/50
1/1 [==============================] - 0s 3ms/step - loss: 5.2187
Epoch 3/50
1/1 [==============================] - 0s 3ms/step - loss: 4.7117
Epoch 4/50
1/1 [==============================] - 0s 5ms/step - loss: 4.2077
Epoch 5/50
1/1 [==============================] - 0s 4ms/step - loss: 3.7066
Epoch 6/50
1/1 [==============================] - 0s 4ms/step - loss: 3.2082
Epoch 7/50
1/1 [==============================] - 0s 5ms/step - loss: 2.7123
Epoch 8/50
1/1 [==============================] - 0s 4ms/step - loss: 2.2188
Epoch 9/50
1/1 [==============================] - 0s 3ms/step - loss: 1.7274
Epoch 10/50
1/1 [==============================] - 0s 5ms/step - loss: 1.2378
Epoch 11/50
1/1 [==============================] - 0s 4ms/step - loss: 0.7497
Epoch 12/50
1/1 [==============================] - 0s 5ms/step - loss: 0.2627
Epoch 13/50
1/1 [==============================] - 0s 4ms/step - loss: 

In [1]:
import numpy as np
import pandas as pd
from keras.layers import Input, Dense
from keras.models import Model

# Load the CSV file
df = pd.read_csv('inflation_annual_percent.csv')
df.fillna(0, inplace=True)

# Transpose the dataframe
df = df.T

# Set the first row as the column headers
df.columns = df.iloc[0]
df = df[1:]

# Replace any negative signs
df = df.replace('−', '-', regex=True)
df.replace('k', '000', inplace=True, regex=True) # remove the 'k' character

df.replace('µ', '', inplace=True, regex=True) # remove the 'k' character

# Define the input shape
input_shape = (60,)

# Define the encoding dimension
encoding_dim = 3

# Define the input layer
input_layer = Input(shape=input_shape)

# Define the encoding layer
encoding_layer = Dense(encoding_dim, activation='relu')(input_layer)

# Define the decoding layer
decoding_layer = Dense(60, activation='sigmoid')(encoding_layer)

# Define the autoencoder model
autoencoder = Model(input_layer, decoding_layer)

# Define the encoder model
encoder = Model(input_layer, encoding_layer)

# Define the decoder model
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Loop through each column in the dataframe
for column in df:
    # Prepare the data for the current column
    data = df[column].to_numpy().astype('float32')
    data = np.reshape(data, (1, -1))  # Reshape to 2D tensor

    # Train the autoencoder model
    autoencoder.fit(data, data, epochs=2)

    # Use the encoder to get the encoded representation of the data
    encoded_data = encoder.predict(data)

    # Use the decoder to get the reconstructed data from the encoded data
    decoded_data = decoder.predict(encoded_data)

    # Print the reconstructed data
    print(column, decoded_data)


Epoch 1/2
1/1 [==============================] - 0s 486ms/step - loss: 1.0284
Epoch 2/2
1/1 [==============================] - 0s 39ms/step
Aruba [[0.69216156 0.536071   0.41728747 0.47555214 0.471064   0.6096484
  0.6478102  0.5536656  0.49400157 0.54624236 0.358596   0.6068458
  0.47485277 0.6464325  0.50407267 0.40768763 0.56647944 0.70704585
  0.5150623  0.27170733 0.5330842  0.4964492  0.3191599  0.3430958
  0.473342   0.5285156  0.47332886 0.51522887 0.69290555 0.42164156
  0.6277187  0.39028233 0.54758507 0.38193715 0.45661402 0.36106682
  0.46567547 0.5934429  0.28576308 0.5593787  0.3099563  0.24983053
  0.6802364  0.39891332 0.6326685  0.37420785 0.5346634  0.4727354
  0.28237975 0.56933326 0.46449745 0.6153877  0.5083371  0.3010392
  0.4857522  0.71943974 0.31382784 0.57584786 0.34783754 0.5020561 ]]
Epoch 1/2
1/1 [==============================] - 0s 6ms/step - loss: 0.7479
Epoch 2/2
1/1 [==============================] - 0s 18ms/step
Afghanistan [[0.7715572  0.7957376  0.3